Use this script to generate an assignment list and roster file from a blackboard gradebook download

In [5]:
!pip install -qq --user openpyxl

In [1]:
import pandas as pd
import requests
import glob
from IPython.display import display, HTML
from ipywidgets import interact_manual, widgets
from datetime import datetime

In [14]:
def get_table(syllabus_url='http://ist256.com/syllabus', table=1):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }
    response = requests.get(syllabus_url, headers=headers)
    html = pd.read_html(response.text)
    return html[table]


def generate_column(gradebook, columns):
    for col in columns:
        if col.startswith(gradebook):
            colid = col.split("|")[-1]
            return gradebook + "|" + colid


def create_assignments_df(a_df, b_df_columns):
    assignments_df = a_df[a_df['Where?'] == 'Jupyterhub'].copy()
    assignments_df['points'] = assignments_df['Points']
    assignments_df['unit_name'] = assignments_df['What is Due?'].apply(
        lambda s: s.split('/')[0]
    )
    assignments_df['assignment_name'] = assignments_df['What is Due?'].apply(
        lambda s: s.split('/')[1]
    )
    assignments_df['due_date'] = assignments_df.apply(
        lambda row: row['Date Due'] + ' ' + row['Time Due'], axis=1
    )
    assignments_df['lms_column_id'] = assignments_df['Gradebook'].apply(
        lambda g: generate_column(g, b_df_columns)
    )
    assignments_df_out = assignments_df[
        ['unit_name', 'assignment_name', 'points', 'due_date', 'lms_column_id']
    ]
    return assignments_df_out


def create_roster_df(b_df, t_df, email_domain="@syr.edu"):
    roster_df = b_df[b_df['Username'].str.find("_previewuser") == -1].copy()
    roster_df['class_no'] = roster_df['Child Course ID'].dropna().astype(int)
    roster_df['student'] = roster_df['Username']
    t_df['instructor'] = t_df['Professor Email'].str.replace(email_domain, "")
    merged_df = pd.merge(
        roster_df, t_df, left_on="class_no", right_on="Class #"
    )
    roster_df_out = merged_df[['student', 'instructor']]
    additions_df = pd.DataFrame({'student': t_df['instructor'].unique(),
                                 'instructor': t_df['instructor'].unique()})
    roster_df_out = pd.concat([roster_df_out, additions_df], ignore_index=True)
    roster_df_out['student'] = roster_df_out.apply(
        lambda row: row['student'] + email_domain, axis=1
    )
    roster_df_out['instructor'] = roster_df_out.apply(
        lambda row: row['instructor'] + email_domain, axis=1
    )
    roster_df_out['grader'] = roster_df_out['instructor']

    return roster_df_out[['student', 'instructor', 'grader']]

In [10]:
email_domain = "@syr.edu"
t_df = get_table(syllabus_url, table=0)
b_df = pd.read_excel("./gradebooks/gc_17008.1251m_fullgc_2024-08-06-12-21-46.xlsx")

roster_df = b_df[b_df['Username'].str.find("_previewuser") == -1].copy()
roster_df['class_no'] = roster_df['Child Course ID'].dropna().astype(int)
roster_df['student'] = roster_df['Username']
t_df['instructor'] = t_df['Professor Email'].str.replace(email_domain, "")
merged_df = pd.merge(roster_df, t_df, left_on="class_no", right_on="Class #")
roster_df_out = merged_df[['student', 'instructor']]
additions_df = pd.DataFrame({'student': t_df['instructor'].unique(), 'instructor': t_df['instructor'].unique()})
roster_df_out = pd.concat([roster_df_out, additions_df], ignore_index=True)
roster_df_out.sample(10)
roster_df_out['student'] = roster_df_out.apply(
    lambda row: row['student'] + email_domain, axis=1
)
roster_df_out['instructor'] = roster_df_out.apply(
    lambda row: row['instructor'] + email_domain, axis=1
)
roster_df_out['grader'] = roster_df_out['instructor']
roster_df_out

/tmp/ipykernel_2026/3265084508.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html = pd.read_html(response.text)


,student,instructor,grader
0,aiabdela@syr.edu,pjagadev@syr.edu,pjagadev@syr.edu
1,ekbazile@syr.edu,pjagadev@syr.edu,pjagadev@syr.edu
2,rebegley@syr.edu,pjagadev@syr.edu,pjagadev@syr.edu
3,ancranda@syr.edu,pjagadev@syr.edu,pjagadev@syr.edu
4,cdavis39@syr.edu,pjagadev@syr.edu,pjagadev@syr.edu
...,...,...,...
96,jsmit163@syr.edu,nschneid@syr.edu,nschneid@syr.edu
97,artrawal@syr.edu,nschneid@syr.edu,nschneid@syr.edu
0,mafudge@syr.edu,mafudge@syr.edu,mafudge@syr.edu
1,pjagadev@syr.edu,pjagadev@syr.edu,pjagadev@syr.edu


In [15]:
files = glob.glob("gradebooks/*.xlsx")
files.sort()
syllabus_url = 'http://ist256.com/syllabus'
data_path = "./data"
email_suffix = "@syr.edu"
files_dropdown = widgets.Dropdown(
            options=files,
            description='Gradebook File:', layout={'width': 'max-content'}
)

@interact_manual(term="ist256-fall2024", gradebook_file=files_dropdown)
def onclick(term, gradebook_file):
    timestamp = gradebook_file.split("_")[-1].replace(".xlsx", "")
    assignments_file = f"{data_path}/{term}-assignments.csv"
    roster_file = f"{data_path}/{term}-roster.csv"
    display(HTML(f"GRADEBOOK FILE: {gradebook_file}"))
    a_df = get_table(syllabus_url, table=-1)
    t_df = get_table(syllabus_url, table=0)
    b_df = pd.read_excel(gradebook_file)
    assignments_df = create_assignments_df(a_df, b_df.columns)
    roster_df = create_roster_df(b_df, t_df)
    assignments_df.to_csv(assignments_file, index=False, header=True)
    roster_df.to_csv(roster_file, index=False, header=True)
    display(HTML(f"ASSIGMENTS FILE: {assignments_file}"))
    display(HTML(f"ROSTER FILE: {roster_file}"))

interactive(children=(Text(value='ist256-fall2024', continuous_update=False, description='term'), Dropdown(des…

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")